<a href="https://colab.research.google.com/github/nstsj/python_for_CL/blob/master/parse_guardian_dz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise: find all news written today on  and parse the content of https://www.theguardian.com/international each of them into a .txt file

стратегия работы:

1. используйте код из тетрадки с пары или из этой (здесь то же самое, но в функциях), чтобы получить лист всех ссылок с главной страницы новостного сайта за определенный день (например, за сегодня)

2. Из листа ссылок сделайте лист со ссылками только на новости (ссылки на катеогрии и другие разделы сайта не нужны)

3. для каждой ссылки на новость в листе ссылок:
    3.1. пройдите по ссылке
    3.2. из этой веб-странички вытащите текст новости + ее заголовок
    3.3. запишите в отдельный .txt файл

4. сохраните файлы в папку

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import re
import os

In [2]:
def parsing_pipe(url):
    req = requests.get(url) #сохраняем ответ сервера на наш реквест
    content = req.content #получаем текст страницы со всеми html-тэгами
    soup = bs(content, 'html.parser')#модуль для html-парсинга: он сделает текст веб-страницы, извлеченный с помощью Requests, более читаемым, потому что создает дерево синтаксического разбора из проанализированных HTML (или XML) документов
    return soup

def get_today_date():
    today = dt.date.today() #фунция генерирует сегодняшнюю дату
    return today.strftime("%Y/%b/%d")
    #https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior for strftime format explanation
    
def parse_link(soup):
    todaynews = []
    listlinks = soup.find_all("a", {"href": True}) #извлечем все ссылки, которые есть на сайте
    for link in listlinks: #Ищем ссылки на сегодняшние новости
        if re.search(get_today_date(),link["href"],re.I) != None:
            todaynews.append(link["href"])
    return set(todaynews)
        

In [4]:
url = "https://www.theguardian.com/international"
pagecontent = parsing_pipe(url)
pagelist = parse_link(pagecontent)
pagelist

{'https://www.theguardian.com/artanddesign/2019/nov/29/austria-feminist-agitator-renaissance-bling-british-museum-the-week-in-art',
 'https://www.theguardian.com/artanddesign/2019/nov/29/james-barnor-ghanaian-photographer-swinging-sixties-independence-black-90',
 'https://www.theguardian.com/books/2019/nov/29/in-the-cut-susanna-moore',
 'https://www.theguardian.com/cities/2019/nov/29/natural-genius-is-to-be-respected-inside-clevelands-space-for-teen-poets',
 'https://www.theguardian.com/cities/2019/nov/29/rubbish-seaside-a-backhanded-love-letter-to-urban-britain',
 'https://www.theguardian.com/commentisfree/2019/nov/29/benin-bronze-sculpture-home-jesus-college-cambridge-return',
 'https://www.theguardian.com/commentisfree/2019/nov/29/black-friday-sale-fashion-clothes-shopping',
 'https://www.theguardian.com/commentisfree/2019/nov/29/britain-history-racism-infects-politics-slavery-windrush',
 'https://www.theguardian.com/commentisfree/2019/nov/29/mass-consumerism-black-friday-climate-ca

In [8]:
def parse_and_write_article(url, path):
    pagesoup = parsing_pipe(url)
    title = pagesoup.find("h1").get_text() #вытаскиваем именно текст, без тэгов 
    text = " ".join([i.get_text() for i in pagesoup.find_all("p")])
    
    with open(path + os.sep + url.rsplit("/", 1)[1] + ".txt", "w+", encoding = "utf-8") as f:
        f.write(title + "\n" + text)        

In [9]:
path = "." + os.sep + dt.date.today().strftime("%Y%m%d")
try:
    os.mkdir(path)
except:
    pass
for link in pagelist:
    pagesoup = parse_and_write_article(link, path)